In [ ]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 7.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import string
import re 
import nltk
import string
import contractions
from copy import deepcopy
import nltk
from nltk.stem import PorterStemmer
import random
from collections import Counter
from termcolor import colored
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from sklearn.model_selection import train_test_split
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
cleaned_data = pd.read_csv('cleaned_data.csv', sep=',', encoding='utf-8-sig', engine='python')

In [ ]:
cleaned_data['dialect'].value_counts()

EG    57636
LY    36499
LB    27617
SD    14434
MA    11539
Name: dialect, dtype: int64

In [ ]:
cleaned_data.isna().sum()

dialect      0
text       292
dtype: int64

In [ ]:
cleaned_data.dropna(inplace=True)

In [ ]:
features = cleaned_data['text']

label = cleaned_data['dialect']


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(label)

In [ ]:
np.unique(label,return_counts = True)

(array([0, 1, 2, 3, 4]), array([57506, 27590, 36446, 11512, 14379]))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=.1, stratify=label, random_state=42)

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
arabic_stopwords = stopwords.words('arabic')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words=arabic_stopwords,ngram_range=(1, 2))
vectorizer.fit(x_train)
x_train_v = vectorizer.transform(x_train)
x_test_v = vectorizer.transform(x_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آمين', 'أب', 'أخ', 'أفعل', 'أفعله', 'ؤلاء', 'إل', 'إم', 'ات', 'اتان', 'ارتد', 'ان', 'انفك', 'برح', 'تان', 'تبد', 'تحو', 'تعل', 'حد', 'حم', 'حي', 'خب', 'ذار', 'سيما', 'صه', 'ظل', 'ظن', 'عد', 'قط', 'مر', 'مكان', 'مكانكن', 'نب', 'هات', 'هب', 'واها', 'وراء'] not in stop_words.
  warnings.warn(


In [ ]:
import pickle
with open("vectorizer.pickle", "wb") as file:
    # Pickle the dictionary using the dump method
    pickle.dump(vectorizer, file)

In [ ]:
with open('vectorizer.pickle','rb') as file:
    tokenizer = pickle.load(file)

tokenizer.transform(["انا حزين"])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آمين', 'أب', 'أخ', 'أفعل', 'أفعله', 'ؤلاء', 'إل', 'إم', 'ات', 'اتان', 'ارتد', 'ان', 'انفك', 'برح', 'تان', 'تبد', 'تحو', 'تعل', 'حد', 'حم', 'حي', 'خب', 'ذار', 'سيما', 'صه', 'ظل', 'ظن', 'عد', 'قط', 'مر', 'مكان', 'مكانكن', 'نب', 'هات', 'هب', 'واها', 'وراء'] not in stop_words.
  warnings.warn(


<1x1297943 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.svm import LinearSVC

svc_model = LinearSVC()
svc_model.fit(x_train_v,y_train)

LinearSVC()

In [ ]:
from sklearn.metrics import classification_report
y_pred = svc_model.predict(x_test_v)
report = classification_report(y_test, y_pred)
print(report)
print("accuracy: {:0.3f}".format(svc_model.score(x_test_v, y_test)))

              precision    recall  f1-score   support

           0       0.82      0.93      0.87      5751
           1       0.87      0.84      0.85      2759
           2       0.82      0.82      0.82      3645
           3       0.85      0.66      0.74      1151
           4       0.85      0.59      0.70      1438

    accuracy                           0.83     14744
   macro avg       0.84      0.77      0.80     14744
weighted avg       0.83      0.83      0.83     14744

accuracy: 0.831


In [ ]:
i = 1
tokens = {}
def tokenize(text):
  global i
  for word in text.split():
    if word not in tokens:
      tokens[word] = i
      i += 1
cleaned_data['text'].apply(tokenize)

0         None
1         None
2         None
3         None
4         None
          ... 
147720    None
147721    None
147722    None
147723    None
147724    None
Name: text, Length: 147433, dtype: object

In [ ]:
voc_size = len(tokens)

In [ ]:
def data_encodeing(text):
  word_list = [tokens[i] for i in text.split()]
  return np.array(word_list)
cleaned_data['encoded_text'] = cleaned_data['text'].apply(data_encodeing)

In [ ]:
cleaned_data.head()

,dialect,text,encoded_text
0,LY,قليلين ادب ومنافقين لو اختهم او قريبتهم تتعاكس...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,LY,الليبيين متقلبين بس بالنسبه ليا انا ميليشياوي ...,"[17, 18, 19, 20, 21, 22, 23, 24, 25]"
2,LY,كل تانيه شاب ليبي بيرتاح لبنت مختلفه ويلاحظ ان...,"[26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 26, 3..."
3,LY,رانيا عقليتك متخلفه اولا الانسان يلي يحتاج اهل...,"[54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 6..."
4,LY,شكلك متعقده علشان الراجل لي تحبيه ازوج بنت يتي...,"[96, 97, 64, 98, 99, 100, 101, 102, 103, 77, 1..."


In [ ]:
m = 0
for i in cleaned_data['encoded_text']:
  if len(i) > m:
    m = len(i)
m

66

In [ ]:
features = np.zeros((cleaned_data['encoded_text'].shape[0], m))
for i, sent in enumerate(cleaned_data['encoded_text']):
    features[i, :len(sent)] = sent[:m]


In [ ]:
import tensorflow as tf

label = tf.keras.utils.to_categorical(label)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.1, random_state=42)

In [ ]:
X_train.shape

(132689, 66)

In [ ]:
y_train.shape

(132689, 5)

In [ ]:
eval = []

In [ ]:
import tensorflow as tf
model = tf.keras.models.Sequential([    
    tf.keras.layers.Embedding(voc_size + 1, 128),
    tf.keras.layers.SimpleRNN(128),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         28261760  
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dense_1 (Dense)             (None, 5)                 645       
                                                                 
Total params: 28,311,813
Trainable params: 28,311,813
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=5, batch_size=128,
                    validation_data=(X_test, y_test), 
                    validation_steps=30)

Epoch 1/5
1037/1037 [==============================] - 150s 138ms/step - loss: 1.3006 - accuracy: 0.4725 - val_loss: 1.3105 - val_accuracy: 0.4839
Epoch 2/5
1037/1037 [==============================] - 89s 86ms/step - loss: 1.2121 - accuracy: 0.5351 - val_loss: 1.2168 - val_accuracy: 0.5456
Epoch 3/5
1037/1037 [==============================] - 81s 78ms/step - loss: 1.2952 - accuracy: 0.4983 - val_loss: 1.3738 - val_accuracy: 0.4695
Epoch 4/5
1037/1037 [==============================] - 80s 77ms/step - loss: 1.3078 - accuracy: 0.4995 - val_loss: 1.3684 - val_accuracy: 0.4740
Epoch 5/5
1037/1037 [==============================] - 81s 78ms/step - loss: 1.3045 - accuracy: 0.5056 - val_loss: 1.3755 - val_accuracy: 0.4742


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
eval.append({'model' : 'SimpleRNN','test_acc' : test_acc,'test_loss' : test_loss })
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

461/461 [==============================] - 6s 13ms/step - loss: 1.0060 - accuracy: 0.6434
Test Loss: 1.0059629678726196
Test Accuracy: 0.6434481739997864


In [ ]:
import tensorflow as tf
model = tf.keras.models.Sequential([    
    tf.keras.layers.Embedding(voc_size + 1, 128),
    tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(128,return_sequences=True)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(128)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 128)         28261760  
                                                                 
 bidirectional_2 (Bidirectio  (None, None, 256)        65792     
 nal)                                                            
                                                                 
 dropout (Dropout)           (None, None, 256)         0         
                                                                 
 bidirectional_3 (Bidirectio  (None, 256)              98560     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_4 (Dense)             (None, 128)              

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=5, batch_size=128,
                    validation_data=(X_test, y_test), 
                    validation_steps=30)

Epoch 1/5
1037/1037 [==============================] - 311s 294ms/step - loss: 0.8717 - accuracy: 0.6766 - val_loss: 0.5792 - val_accuracy: 0.8010
Epoch 2/5
1037/1037 [==============================] - 264s 255ms/step - loss: 0.4055 - accuracy: 0.8697 - val_loss: 0.5733 - val_accuracy: 0.8034
Epoch 3/5
1037/1037 [==============================] - 274s 265ms/step - loss: 0.2316 - accuracy: 0.9303 - val_loss: 0.6852 - val_accuracy: 0.8018
Epoch 4/5
1037/1037 [==============================] - 262s 253ms/step - loss: 0.1592 - accuracy: 0.9524 - val_loss: 0.7208 - val_accuracy: 0.7958
Epoch 5/5
1037/1037 [==============================] - 270s 260ms/step - loss: 0.1163 - accuracy: 0.9654 - val_loss: 0.7960 - val_accuracy: 0.7945


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
eval.append({'model' : 'biDirectionalSimpleRNN','test_acc' : test_acc,'test_loss' : test_loss })
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

461/461 [==============================] - 11s 24ms/step - loss: 0.7817 - accuracy: 0.7977
Test Loss: 0.7817309498786926
Test Accuracy: 0.7976804375648499


In [ ]:
model = tf.keras.models.Sequential([    
    tf.keras.layers.Embedding(voc_size + 1, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=5, batch_size=128,
                    validation_data=(X_test, y_test), 
                    validation_steps=30)

Epoch 1/5
1037/1037 [==============================] - 95s 83ms/step - loss: 0.6488 - accuracy: 0.7640 - val_loss: 0.4799 - val_accuracy: 0.8313
Epoch 2/5
1037/1037 [==============================] - 28s 27ms/step - loss: 0.2471 - accuracy: 0.9203 - val_loss: 0.5249 - val_accuracy: 0.8227
Epoch 3/5
1037/1037 [==============================] - 26s 25ms/step - loss: 0.1169 - accuracy: 0.9637 - val_loss: 0.6109 - val_accuracy: 0.8172
Epoch 4/5
1037/1037 [==============================] - 23s 22ms/step - loss: 0.0721 - accuracy: 0.9780 - val_loss: 0.7186 - val_accuracy: 0.8023
Epoch 5/5
1037/1037 [==============================] - 21s 20ms/step - loss: 0.0508 - accuracy: 0.9844 - val_loss: 0.8073 - val_accuracy: 0.8003


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
eval.append({'model' : 'GRU_RNN','test_acc' : test_acc,'test_loss' : test_loss })

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

461/461 [==============================] - 3s 7ms/step - loss: 0.8212 - accuracy: 0.8047
Test Loss: 0.8212136626243591
Test Accuracy: 0.8047341108322144


In [ ]:
pd.DataFrame(eval)

,model,test_acc,test_loss
0,biDirectionalSimpleRNN,0.797680,0.781731
1,GRU_RNN,0.804734,0.821214
2,GRU_RNN,0.804734,0.821214


## the Machine Learning model had better performance than the more complex Deep learning model so it is going to the Deployment .....yay!!!!

In [ ]:
from joblib import dump

# Save the model
dump(svc_model, "my_model.joblib")

['my_model.joblib']